In [1]:
import pandas as pd
data_path = "../../data/open_source_8454_combine_short_description.csv"  # 替換為你的檔案路徑
df = pd.read_csv(data_path)
# df_label_0 = df[df["Label"] == 0]
# df_label_0_to_drop = df_label_0.sample(n=3000, random_state=42)
# df = df.drop(df_label_0_to_drop.index)
df

,Inbound Message,Label
0,login issue verified user details employee man...,0
1,outlook hello team meetings skype meetings etc...,0
2,cant log vpn cannot log vpn best,0
3,unable access tool page unable access tool page,0
4,skype error skype error,0
...,...,...
8449,emails coming mail good afternoon receiving em...,22
8450,telephony software issue telephony software issue,0
8451,vip windows password reset tifpdchb pedxruyf v...,0
8452,machine est funcionando unable access machine ...,44


In [2]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}") 
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
print(f"Current GPU Memory Allocated: {torch.cuda.memory_allocated() / 1e9} GB")
print(f"Current GPU Memory Cached: {torch.cuda.memory_reserved() / 1e9} GB")

Using device: cuda
CUDA Available: True
CUDA Device: NVIDIA GeForce RTX 3050 6GB Laptop GPU
Current GPU Memory Allocated: 0.0 GB
Current GPU Memory Cached: 0.0 GB


In [3]:
from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, messages, labels, tokenizer, max_length):
        self.messages = messages
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, idx):
        message = str(self.messages[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer.encode_plus(
            message,
            max_length=self.max_length,
            add_special_tokens=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [4]:
# 定義訓練與評估函數
def train_epoch(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader):
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs["loss"]
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    return total_loss / len(dataloader)

In [5]:
from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef
from sklearn.preprocessing import label_binarize
import numpy as np
import torch

def eval_model(model, dataloader, criterion, num_labels):
    model.eval()
    total_loss = 0
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs["logits"]
            loss = criterion(logits, labels)
            total_loss += loss.item()

            probs = torch.softmax(logits, dim=1)
            _, preds = torch.max(logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    val_loss = total_loss / len(dataloader)
    val_accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    mcc = matthews_corrcoef(all_labels, all_preds)

    return {
        "val_loss": val_loss,
        "val_accuracy": val_accuracy,
        "precision": precision,
        "recall": recall,
        "f1-score": f1,
        "macro_f1-score": macro_f1,
        "balanced_accuracy": balanced_acc,
        "mcc": mcc
    }

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import StratifiedKFold
# 設定參數
MAX_LENGTH = 128
BATCH_SIZE = 16
EPOCHS = 20

# 初始化 BERT 的 tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 初始化數據
messages = df['Inbound Message'].tolist()
labels = df['Label'].tolist()

# KFold 初始化
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [7]:
from transformers import BertModel
import torch.nn as nn

class BertWithLSTM(nn.Module):
    def __init__(self, num_labels, hidden_size=768, lstm_hidden_size=256, lstm_layers=1, dropout=0.3):
        super(BertWithLSTM, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.lstm = nn.LSTM(hidden_size, lstm_hidden_size, num_layers=lstm_layers,
                            batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(lstm_hidden_size * 2, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        sequence_output = outputs.last_hidden_state  # (batch_size, seq_len, hidden_size)
        lstm_out, _ = self.lstm(sequence_output)     # (batch_size, seq_len, lstm_hidden_size*2)

        # 使用 masked mean pooling
        mask = attention_mask.unsqueeze(-1).expand(lstm_out.size())
        masked_lstm_out = lstm_out * mask
        summed = torch.sum(masked_lstm_out, 1)
        counted = torch.clamp(mask.sum(1), min=1e-9)
        pooled = summed / counted  # (batch_size, lstm_hidden_size*2)

        logits = self.classifier(self.dropout(pooled))
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)
            return {"loss": loss, "logits": logits}
        else:
            return {"logits": logits}

In [9]:
from torch.utils.data import DataLoader
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import AdamW
from tqdm import tqdm
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

all_fold_results = []

# 進行 10 折交叉驗證
for fold, (train_idx, val_idx) in enumerate(skf.split(messages, labels)):
    print(f"\n===== Fold {fold + 1} =====")
    train_messages = [messages[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_messages = [messages[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]

    print("Train label dist:", {l: train_labels.count(l) for l in set(train_labels)})
    print("Val label dist:", {l: val_labels.count(l) for l in set(val_labels)})

    # 計算各 Label 的數量
    label_counts = pd.Series(train_labels).value_counts()
    avg_label_size = int(label_counts.mean())  # **取平均值作為 resample 基準點**
    print(f"avg_label_size: {avg_label_size}")

    # 找出需要過採樣的類別
    labels_to_oversample = label_counts[label_counts < avg_label_size].index.tolist()
    
    # 進行 ROS (Random Oversampling)
    ros = RandomOverSampler(sampling_strategy={label: avg_label_size for label in labels_to_oversample}, random_state=42)
    train_messages_resampled, train_labels_resampled = ros.fit_resample(np.array(train_messages).reshape(-1, 1), train_labels)
    train_messages_resampled = train_messages_resampled.flatten().tolist()

    # 轉換成 DataLoader
    train_dataset = CustomDataset(train_messages_resampled, train_labels_resampled, tokenizer, MAX_LENGTH)
    val_dataset = CustomDataset(val_messages, val_labels, tokenizer, MAX_LENGTH)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    NUM_LABELS = len(set(train_labels) | set(val_labels))
    model = BertWithLSTM(num_labels=NUM_LABELS).to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=1e-2)
    criterion = CrossEntropyLoss()

    epoch_results = []
    patience = 3
    best_macro_f1 = 0
    epochs_without_improvement = 0

    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch + 1}/{EPOCHS}")
        train_loss = train_epoch(model, train_dataloader, optimizer, criterion)
        print(f"Train loss: {train_loss:.4f}")
        metrics = eval_model(model, val_dataloader, criterion, num_labels=NUM_LABELS)
        print(metrics)
        epoch_results.append(metrics)

        current_macro_f1 = metrics["macro_f1-score"]
        if current_macro_f1 > best_macro_f1:
            best_macro_f1 = current_macro_f1
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"Early stopping triggered at epoch {epoch + 1}")
            break

    avg_metrics = {k: np.mean([e[k] for e in epoch_results]) for k in epoch_results[0].keys()}
    all_fold_results.append(avg_metrics)

# ===== 6. 結果統整 =====
final_metrics = {k: np.mean([f[k] for f in all_fold_results]) for k in all_fold_results[0].keys()}
print("\n=== Final 10-fold Cross-Validation Results ===")
for metric, value in final_metrics.items():
    print(f"{metric}: {value:.4f}")

D:\Anaconda3\envs\cuda\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 7 members, which is less than n_splits=10.
  warnings.warn(



===== Fold 1 =====
Train label dist: {0: 3576, 1: 28, 2: 126, 3: 27, 4: 231, 5: 130, 6: 107, 7: 35, 8: 76, 9: 73, 10: 79, 11: 194, 12: 217, 13: 32, 14: 26, 15: 28, 16: 23, 17: 260, 18: 104, 19: 51, 20: 16, 21: 39, 22: 88, 23: 180, 24: 18, 25: 56, 26: 97, 27: 55, 28: 13, 29: 15, 30: 17, 31: 90, 32: 40, 33: 36, 34: 34, 35: 13, 36: 32, 37: 24, 38: 7, 39: 116, 40: 13, 41: 9, 42: 166, 43: 18, 44: 22, 45: 10, 46: 61, 47: 595, 48: 227, 49: 78}
Val label dist: {0: 398, 1: 3, 2: 14, 3: 3, 4: 26, 5: 15, 6: 11, 7: 4, 8: 9, 9: 8, 10: 9, 11: 21, 12: 24, 13: 4, 14: 3, 15: 3, 16: 2, 17: 29, 18: 12, 19: 5, 20: 2, 21: 5, 22: 9, 23: 20, 24: 2, 25: 7, 26: 10, 27: 6, 28: 2, 29: 1, 30: 2, 31: 10, 32: 5, 33: 4, 34: 3, 35: 2, 36: 3, 37: 3, 39: 13, 40: 1, 41: 2, 42: 18, 43: 2, 44: 3, 45: 1, 46: 7, 47: 66, 48: 25, 49: 9}
avg_label_size: 152

Epoch 1/20


100%|██████████| 743/743 [05:28<00:00,  2.26it/s]


Train loss: 2.0921
{'val_loss': 1.4376675745226302, 'val_accuracy': 0.6229314420803782, 'precision': 0.5565755439517769, 'recall': 0.6229314420803782, 'f1-score': 0.558827181905549, 'macro_f1-score': 0.2720805024698875, 'balanced_accuracy': 0.2860550433114697, 'mcc': 0.4716480126219055}

Epoch 2/20


100%|██████████| 743/743 [05:28<00:00,  2.26it/s]


...

Train loss: 0.3096


D:\Anaconda3\envs\cuda\Lib\site-packages\sklearn\metrics\_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


{'val_loss': 1.4561707298710662, 'val_accuracy': 0.693491124260355, 'precision': 0.689216640475439, 'recall': 0.693491124260355, 'f1-score': 0.674254203565171, 'macro_f1-score': 0.39513452943949723, 'balanced_accuracy': 0.39197672430930486, 'mcc': 0.586723459775166}

Epoch 7/20


100%|██████████| 743/743 [06:11<00:00,  2.00it/s]


Train loss: 0.2800
{'val_loss': 1.5399778305359606, 'val_accuracy': 0.6781065088757396, 'precision': 0.6742686364591226, 'recall': 0.6781065088757396, 'f1-score': 0.6602519328680181, 'macro_f1-score': 0.40878340488982806, 'balanced_accuracy': 0.39522571480142066, 'mcc': 0.5736665710031522}
Early stopping triggered at epoch 7

=== Final 10-fold Cross-Validation Results ===
val_loss: 1.4301
val_accuracy: 0.6625
precision: 0.6543
recall: 0.6625
f1-score: 0.6391
macro_f1-score: 0.3766
balanced_accuracy: 0.3870
mcc: 0.5472


In [10]:
all_fold_results

[{'val_loss': 1.4706989445916887,
  'val_accuracy': 0.6653368794326241,
  'precision': 0.6529105061742719,
  'recall': 0.6653368794326241,
  'f1-score': 0.6357176870367138,
  'macro_f1-score': 0.37538812006287986,
  'balanced_accuracy': 0.38247423975921757,
  'mcc': 0.5453340621878493},
 {'val_loss': 1.3673301673722718,
  'val_accuracy': 0.649645390070922,
  'precision': 0.6210041229478882,
  'recall': 0.649645390070922,
  'f1-score': 0.6132052765430069,
  'macro_f1-score': 0.35131209308826017,
  'balanced_accuracy': 0.3685890684078624,
  'mcc': 0.5244632399111969},
 {'val_loss': 1.5042257640721663,
  'val_accuracy': 0.659692671394799,
  'precision': 0.6530613426764613,
  'recall': 0.659692671394799,
  'f1-score': 0.6411562308031187,
  'macro_f1-score': 0.3988878110789694,
  'balanced_accuracy': 0.41798974196099825,
  'mcc': 0.5467717355229773},
 {'val_loss': 1.4685725820889262,
  'val_accuracy': 0.655371683740478,
  'precision': 0.6442475154243629,
  'recall': 0.655371683740478,
  'f1